In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import json

class CafeteriaSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Cafeteria System")
        self.root.geometry("1000x700")
        self.root.configure(bg='#f0f0f0')
        
        # Language settings
        self.current_language = "en"
        # Interface text
        self.text = {
            "title": "Cafeteria System",
            "menu_title": "Menu",
            "cart_title": "Shopping Cart",
            "total": "Total",
            "clear_cart": "Clear Cart",
            "checkout": "Checkout",
            "quantity": "Qty",
            "price": "Price",
            "add_to_cart": "Add to Cart",
            "item": "Item",
            "remove": "Remove",
            "order_complete": "Order Complete!",
            "order_message": "Your order has been processed successfully!",
            "empty_cart": "Cart is empty!",
            "categories": {
                "mains": "Main Dishes",
                "sides": "Sides",
                "beverages": "Beverages",
                "desserts": "Desserts"
            }
        }
        
        # Menu items
        self.menu_items = {
            "mains": [
                {"name": "Grilled Chicken", "price": 12.99},
                {"name": "Beef Burger", "price": 10.99},
                {"name": "Fish & Chips", "price": 14.99},
                {"name": "Pasta Carbonara", "price": 11.99},
                {"name": "Vegetarian Pizza", "price": 13.99}
            ],
            "sides": [
                {"name": "French Fries", "price": 4.99},
                {"name": "Onion Rings", "price": 5.99},
                {"name": "Garden Salad", "price": 6.99},
                {"name": "Mashed Potatoes", "price": 4.99},
                {"name": "Coleslaw", "price": 3.99}
            ],
            "beverages": [
                {"name": "Coca Cola", "price": 2.99},
                {"name": "Orange Juice", "price": 3.99},
                {"name": "Coffee", "price": 2.49},
                {"name": "Iced Tea", "price": 2.99},
                {"name": "Water", "price": 1.99}
            ],
            "desserts": [
                {"name": "Chocolate Cake", "price": 6.99},
                {"name": "Ice Cream", "price": 4.99},
                {"name": "Apple Pie", "price": 5.99},
                {"name": "Cheesecake", "price": 7.99},
                {"name": "Cookies", "price": 3.99}
            ]
        }
        
        # Shopping cart
        self.cart = []
        self.cart_total = 0.0
        
        # Initialize GUI
        self.setup_gui()
        
    def setup_gui(self):
        # Main frame
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Header
        header_frame = tk.Frame(main_frame, bg='#2c3e50', height=80)
        header_frame.pack(fill=tk.X, pady=(0, 20))
        header_frame.pack_propagate(False)
        
        self.title_label = tk.Label(header_frame, text=self.text["title"], 
                                   font=('Arial', 24, 'bold'), fg='white', bg='#2c3e50')
        self.title_label.pack(side=tk.LEFT, padx=20, pady=20)
        
        # Main content frame
        content_frame = tk.Frame(main_frame, bg='#f0f0f0')
        content_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left side - Menu
        menu_frame = tk.LabelFrame(content_frame, text=self.text["menu_title"], 
                                  font=('Arial', 16, 'bold'), bg='#ecf0f1', fg='#2c3e50')
        menu_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 10))
        
        # Menu notebook for categories
        self.menu_notebook = ttk.Notebook(menu_frame)
        self.menu_notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Right side - Shopping Cart
        cart_frame = tk.LabelFrame(content_frame, text=self.text["cart_title"], 
                                  font=('Arial', 16, 'bold'), bg='#ecf0f1', fg='#2c3e50',
                                  width=400)
        cart_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=(10, 0))
        cart_frame.pack_propagate(False)
        
        # Cart listbox
        self.cart_listbox = tk.Listbox(cart_frame, font=('Arial', 10), height=15)
        self.cart_listbox.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Cart buttons frame
        cart_buttons_frame = tk.Frame(cart_frame, bg='#ecf0f1')
        cart_buttons_frame.pack(fill=tk.X, padx=10, pady=5)
        
        self.remove_btn = tk.Button(cart_buttons_frame, text=self.text["remove"],
                                   command=self.remove_from_cart, bg='#e74c3c', fg='white',
                                   font=('Arial', 10, 'bold'))
        self.remove_btn.pack(side=tk.LEFT, padx=5)
        
        self.clear_btn = tk.Button(cart_buttons_frame, text=self.text["clear_cart"],
                                  command=self.clear_cart, bg='#f39c12', fg='white',
                                  font=('Arial', 10, 'bold'))
        self.clear_btn.pack(side=tk.LEFT, padx=5)
        
        # Total and checkout
        total_frame = tk.Frame(cart_frame, bg='#ecf0f1')
        total_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.total_label = tk.Label(total_frame, text=f"{self.text['total']}: $0.00",
                                   font=('Arial', 14, 'bold'), bg='#ecf0f1', fg='#2c3e50')
        self.total_label.pack(pady=5)
        
        self.checkout_btn = tk.Button(total_frame, text=self.text["checkout"],
                                     command=self.checkout, bg='#27ae60', fg='white',
                                     font=('Arial', 12, 'bold'), height=2)
        self.checkout_btn.pack(fill=tk.X, pady=5)
        
        # Create menu tabs
        self.create_menu_tabs()
        
    def create_menu_tabs(self):
        # Clear existing tabs
        for tab in self.menu_notebook.tabs():
            self.menu_notebook.forget(tab)
        
        # Create tabs for each category
        for category in self.menu_items:
            tab_frame = tk.Frame(self.menu_notebook, bg='#ecf0f1')
            self.menu_notebook.add(tab_frame, text=self.text["categories"][category])
            
            # Create scrollable frame for items
            canvas = tk.Canvas(tab_frame, bg='#ecf0f1')
            scrollbar = ttk.Scrollbar(tab_frame, orient="vertical", command=canvas.yview)
            scrollable_frame = tk.Frame(canvas, bg='#ecf0f1')
            
            scrollable_frame.bind(
                "<Configure>",
                lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
            )
            
            canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
            canvas.configure(yscrollcommand=scrollbar.set)
            
            # Add items to scrollable frame
            items = self.menu_items[category]
            for i, item in enumerate(items):
                self.create_item_widget(scrollable_frame, item, i)
            
            canvas.pack(side="left", fill="both", expand=True)
            scrollbar.pack(side="right", fill="y")
    
    def create_item_widget(self, parent, item, row):
        item_frame = tk.Frame(parent, bg='white', relief=tk.RAISED, bd=2)
        item_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # Item name and price
        info_frame = tk.Frame(item_frame, bg='white')
        info_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=10, pady=10)
        
        name_label = tk.Label(info_frame, text=item["name"], font=('Arial', 12, 'bold'),
                             bg='white', fg='#2c3e50')
        name_label.pack(anchor=tk.W)
        
        price_label = tk.Label(info_frame, text=f"${item['price']:.2f}", 
                              font=('Arial', 11), bg='white', fg='#27ae60')
        price_label.pack(anchor=tk.W)
        
        # Controls frame
        controls_frame = tk.Frame(item_frame, bg='white')
        controls_frame.pack(side=tk.RIGHT, padx=10, pady=10)
        
        # Quantity label
        qty_label = tk.Label(controls_frame, text=self.text["quantity"], 
                            font=('Arial', 10), bg='white')
        qty_label.pack()
        
        # Spinbox for quantity
        quantity_var = tk.StringVar(value="1")
        spinbox = tk.Spinbox(controls_frame, from_=1, to=20, width=5, 
                           textvariable=quantity_var, font=('Arial', 10))
        spinbox.pack(pady=2)
        
        # Add to cart button
        add_btn = tk.Button(controls_frame, text=self.text["add_to_cart"],
                           command=lambda: self.add_to_cart(item, quantity_var.get()),
                           bg='#3498db', fg='white', font=('Arial', 9, 'bold'))
        add_btn.pack(pady=5)
    
    def add_to_cart(self, item, quantity):
        try:
            qty = int(quantity)
            if qty > 0:
                cart_item = {
                    "name": item["name"],
                    "price": item["price"],
                    "quantity": qty,
                    "subtotal": item["price"] * qty
                }
                self.cart.append(cart_item)
                self.update_cart_display()
        except ValueError:
            messagebox.showerror("Error", "Invalid quantity")
    
    def update_cart_display(self):
        self.cart_listbox.delete(0, tk.END)
        self.cart_total = 0.0
        
        for i, item in enumerate(self.cart):
            display_text = f"{item['name']} x{item['quantity']} - ${item['subtotal']:.2f}"
            self.cart_listbox.insert(tk.END, display_text)
            self.cart_total += item['subtotal']
        
        self.total_label.config(text=f"{self.text['total']}: ${self.cart_total:.2f}")
    
    def remove_from_cart(self):
        selection = self.cart_listbox.curselection()
        if selection:
            index = selection[0]
            del self.cart[index]
            self.update_cart_display()
    
    def clear_cart(self):
        self.cart.clear()
        self.update_cart_display()
    
    def checkout(self):
        if not self.cart:
            messagebox.showwarning("Warning", self.text["empty_cart"])
            return
        
        # Process order (in a real system, this would handle payment)
        order_details = {
            "items": self.cart,
            "total": self.cart_total,
            "timestamp": datetime.now().isoformat()
        }
        
        messagebox.showinfo(self.text["order_complete"], 
                           self.text["order_message"])
        
        # Clear cart after successful order
        self.clear_cart()
    
    def toggle_language(self):
        self.current_language = "es" if self.current_language == "en" else "en"
        self.refresh_interface()
    
    def refresh_interface(self):
        # Update all text elements
        self.title_label.config(text=self.get_text("title"))
        self.language_btn.config(text=self.get_text("language_btn"))
        
        # Update cart frame
        for widget in self.root.winfo_children():
            if isinstance(widget, tk.Frame):
                for child in widget.winfo_children():
                    if isinstance(child, tk.LabelFrame):
                        if "menu" in child.cget("text").lower() or "menú" in child.cget("text").lower():
                            child.config(text=self.get_text("menu_title"))
                        elif "cart" in child.cget("text").lower() or "carrito" in child.cget("text").lower():
                            child.config(text=self.get_text("cart_title"))
        
        # Update buttons
        self.remove_btn.config(text=self.get_text("remove"))
        self.clear_btn.config(text=self.get_text("clear_cart"))
        self.checkout_btn.config(text=self.get_text("checkout"))
        
        # Update total label
        self.total_label.config(text=f"{self.get_text('total')}: ${self.cart_total:.2f}")
        
        # Recreate menu tabs with new language
        self.create_menu_tabs()
    
    def get_text(self, key):
        return self.translations[self.current_language][key]

if __name__ == "__main__":
    root = tk.Tk()
    app = CafeteriaSystem(root)
    root.mainloop()

: 